In [1]:
import math
#import numpy
import autograd.numpy as np
import random
from autograd import elementwise_grad
from scipy.special import comb
import itertools

In [52]:
state = np.array([1.0, 1, 1, 1, 1, 1, 1, 1, 1])
numcards = 8
ownCards = np.array([0.0, 0, 0, 0, 0, 0, 0, 0, 0])
factor = 1
bluffEpsilon = 0.1  #remember to implement
callsList = np.array([[-1, -1, -1, -1, -1],
                  [ 0, -1, -1, -1, -1],
                  [ 1, -1,  2, -1, -1],
                  [ 3, -1,  4, -1, -1],
                  [ 5, -1, -1,  6, -1]])

def findCall(num):
    for i in range(callsList.size):
        for j in range(callsList[0].size):
            if callsList[i][j] == num:
                return [i, j]



In [4]:
def oneCardProbabilities(state):
    total = np.sum(state)
    oneCardProb = (state+1)/(total+1)
    return oneCardProb

In [40]:
def twotypeCall(state, numcards, countOne, countTwo, ownCards, typeOne, typeTwo):
    return twotype(state, numcards, countOne, countTwo, ownCards)[typeOne][typeTwo]

def twotype(state, numcards, countOne, countTwo, ownCards):
    prob = oneCardProbabilities(state)
    #probPairs = np.array(np.meshgrid(prob,prob)) # so that it has shape (2,13,13)
    probPairs = np.transpose(np.reshape(np.array(list(map(np.array,itertools.product(prob,prob)))),(state.size,state.size,2)),(2,0,1))
    
             #print(probPairs.shape)
#     probabilities = np.zeroes([len(state), len(state)])
#     for i in range(len(state)):
#         for j in range(len(state)):
#             twopairprob(probPairs[i, j], numcards)
    
    return twotypeprob(probPairs, numcards, countOne, countTwo, ownCards)

def twotypeprob(p, numcards, countCardOne, countCardTwo, ownCards):
    prob = 0
    for i in range(countCardOne, 5):
        for j in range(countCardTwo, 5):
            prob = prob + comb(numcards, i-ownCards) * comb(numcards-i+ownCards, j-ownCards) * (p[0])**(i-ownCards)*(p[1])**(j-ownCards)*(1-(p[0]+p[1]))**(numcards-(i+j-2*ownCards))
    return prob
            

def onetypeCall(state, numcards, count, ownCards, cardType):
    return onetype(state, numcards, count, ownCards)[cardType]
    
def onetype(state, numcards, count, ownCards):
    prob = oneCardProbabilities(state)
    return onetypeprob(prob, numcards, count, ownCards)
    
def onetypeprob(p, numcards, countCard, ownCards):
    prob = 0
    for i in range(countCard, 5):
        prob = prob + comb(numcards, i-ownCards)* p**(i-ownCards) * (1-p)**(numcards-i+ownCards)
    return prob

In [6]:
pgrad = elementwise_grad(twotypeCall)

In [58]:
pgrad(state, numcards, 2, 2, ownCards, 0, 4)

array([ 0.08139474, -0.06010993, -0.06010993, -0.06010993,  0.08139474,
       -0.06010993, -0.06010993, -0.06010993, -0.06010993])

In [7]:
pograd = elementwise_grad(onetypeCall)

In [10]:
pograd(state, numcards, 2, ownCards, 4)

array([ 0.05872026,  0.05872026,  0.05872026,  0.05872026, -0.23488102,
        0.05872026,  0.05872026,  0.05872026,  0.05872026])

In [8]:
def updateParameters(call, state, factor, numcards, ownCards):
    if(call[1] == 0):
        grad = pograd(state, numcards, ownCards,  call[0], call[2])
    else:
        grad= pgrad(state, numcards, call[0], call[1], ownCards, call[2], call[3])
    
    return state + factor*grad
    

In [56]:
state

array([1.07913472, 0.94724352, 0.94724352, 0.94724352, 1.07913472,
       0.94724352, 0.94724352, 0.94724352, 0.94724352])

In [55]:
state = updateParameters(np.array([2, 2, 0, 4]), state, factor, numcards, ownCards)


In [57]:
twotypeCall(state, numcards, 3, 2, ownCards, 0, 4)

0.08240618917783003

In [40]:
pgrad(state, numcards, 3, 2, 0, 4)

TypeError: twotypeCall() missing 1 required positional argument: 'typeTwo'

In [9]:
def makeCall(call, state, numcards, ownCards, calls):  #not quite
    allCallProb = [onetype(state, numcards, 1, ownCards), onetype(state, numcards, 2, ownCards), twotype(state, numcards, 2, 2, ownCards), onetype(state, numcards, 3, ownCards), twotype(state, numcards, 3, 2, ownCards), onetype(state, numcards, 4, ownCards), twotype(state, numcards, 4, 3, ownCards)]    
    ind = calls[call[0]][call[1]]
    bestCall = [-1, -1, -1, -1, 0]
    for i in range(ind+1, len(allCallProb)):  #call strength
        for j in range(allCallProb[i].shape[0]):  #level within call
            if len(allCallProb[i].shape) > 1:
                for k in range(allCallProb[i][j].shape[0]):
                    prob = allCallProb[i][j][k]
                    if(prob > bestCall[4] and k!=j):
                        newcall = findCall(i)
                        bestCall = [newcall[0], newcall[1], j, k, prob]
                    
            else:
                
                prob = allCallProb[i][j]
                if(prob > bestCall[4]):
                    newcall = findCall(i)
                    bestCall = [newcall[0], newcall[1], j, -1, prob]
                    
    currLevelProb = allCallProb[ind]
    
    for i in range(call[2], currLevelProb.shape[0]):
        if len(currLevelProb.shape) > 1:
            for j in range(currLevelProb.shape[1]):
                if(i != call[2] or j > call[3]):
                    prob = currLevelProb[i][j]
                    if(prob > bestCall[4] and i != j):
                        
                        newcall = findCall(ind)
                        
                        bestCall = [newcall[0], newcall[1], i, j, prob]
        else:
            if(i != call[2]):
                prob = currLevelProb[i]
                if(prob > bestCall[4]):
                    newcall = findCall(ind)
                    bestCall = [newcall[0], newcall[1], i, -1, prob]
    return bestCall
    

In [44]:
def makeDecision(call, state, numcards, ownCards, calls):
    #somewhat unnecessary
    allCallProb = [onetype(state, numcards, 1, ownCards), onetype(state, numcards, 2, ownCards), twotype(state, numcards, 2, 2, ownCards), onetype(state, numcards, 3, ownCards), twotype(state, numcards, 3, 2, ownCards), onetype(state, numcards, 4, ownCards), twotype(state, numcards, 4, 3, ownCards)]

    ownCall = makeCall(call, state, numcards, ownCards, calls)
    oppCallProb = 0
    if(call[1] == 0):
        oppCallProb = allCallProb[calls[call[0]][call[1]]][call[2]]
    else:
       
        oppCallProb = allCallProb[calls[call[0]][call[1]]][call[2]][call[3]]
    print(oppCallProb)
    if ownCall[4] > 1-oppCallProb:
        return 1
    else:
        return 0

In [20]:
state

array([1., 1., 1., 1., 1., 1., 1., 1., 1.])

In [52]:
x = np.array([1,2])

In [53]:
state

array([1.00072041, 1.99994642, 2.99994642, 3.99994642, 5.00009923,
       5.99994642, 6.99994642, 7.99994642, 8.99994642])

In [140]:
makeCall(np.array([3, 2, 2, 1]), state, numcards, calls)

[3 2 2 1]


[3, 2, 3, 1, 0.49742688497945775]

In [23]:
makeDecision(np.array([3,2,2,1]), state, numcards, calls)

IndexError: list index out of range

In [24]:
allCallProb

NameError: name 'allCallProb' is not defined

NameError: name 'prob' is not defined

In [26]:
prob = oneCardProbabilities(state)

In [27]:
prob

array([0.0434935 , 0.0652156 , 0.08695452, 0.10869344, 0.13043568,
       0.15217128, 0.1739102 , 0.19564912, 0.21738804])

In [11]:
def runGame():
    bluffCalled = 0

    numPlayerCards = 4
    numCpuCards = 4
    deckSize = 13
    
    deck = [i for i in range(deckSize) for j in range(4)]
    random.shuffle(deck)
    playerCards=deck[0:4]
    cpuCards = deck[4:8]
    
    
    state = np.array([1 for i in range(deckSize)])
    ownCards = np.array([0 for i in range(deckSize)])
    for i in cpuCards:
        ownCards[i] = ownCards[i]+1
        
    print('These are your cards:', playerCards)
    
    while not bluffCalled:
        
        userIn = input('Make your decision:')
        if(userIn == "bluff"):
            bluffCalled = 1
            break
        else:
            call= [int(userIn[0]), int(userIn[2]), int(userIn[4]), int(userIn[6])]
            decision = makeDecision(call, state, numPlayerCards, ownCards, callsList)
            if not decision:
                print("Bluff")
                bluffCalled = 1
            else:
                print(makeCall(call, state, numPlayerCards, ownCards, callsList))
            
            
        
            
            
    
    

In [34]:
pgrad(state, numcards, 4, 3, 1, 2)

array([-4.31194257e-07,  3.35733436e-06,  1.68627027e-06, -4.31194257e-07,
       -4.31194257e-07, -4.31194257e-07, -4.31194257e-07, -4.31194257e-07,
       -4.31194257e-07])

In [35]:
3**np.array([1, 2, 3])

array([ 3,  9, 27], dtype=int32)

In [72]:
calls[0].size

5

In [92]:
prob = oneCardProbabilities(state)
np.array(list(map(np.array,itertools.product(prob,prob))))

array([[0.04347826, 0.04347826],
       [0.04347826, 0.06521739],
       [0.04347826, 0.08695652],
       [0.04347826, 0.10869565],
       [0.04347826, 0.13043478],
       [0.04347826, 0.15217391],
       [0.04347826, 0.17391304],
       [0.04347826, 0.19565217],
       [0.04347826, 0.2173913 ],
       [0.06521739, 0.04347826],
       [0.06521739, 0.06521739],
       [0.06521739, 0.08695652],
       [0.06521739, 0.10869565],
       [0.06521739, 0.13043478],
       [0.06521739, 0.15217391],
       [0.06521739, 0.17391304],
       [0.06521739, 0.19565217],
       [0.06521739, 0.2173913 ],
       [0.08695652, 0.04347826],
       [0.08695652, 0.06521739],
       [0.08695652, 0.08695652],
       [0.08695652, 0.10869565],
       [0.08695652, 0.13043478],
       [0.08695652, 0.15217391],
       [0.08695652, 0.17391304],
       [0.08695652, 0.19565217],
       [0.08695652, 0.2173913 ],
       [0.10869565, 0.04347826],
       [0.10869565, 0.06521739],
       [0.10869565, 0.08695652],
       [0.

In [134]:
twotype(state, numcards, 3, 2, ownCards)

array([[7.18950219e-05, 6.23392158e-01, 5.36118909e-01, 4.58983086e-01,
        3.91092877e-01, 6.94962101e-04, 8.64181184e-04, 1.04062577e-03,
        1.22154318e-03],
       [2.30097998e-04, 5.86788684e-01, 5.02862922e-01, 4.28940575e-01,
        3.64107782e-01, 2.21449261e-03, 2.75110885e-03, 3.30960438e-03,
        3.88111767e-03],
       [5.16600830e-04, 5.43934449e-01, 4.64385244e-01, 3.94567414e-01,
        3.33559691e-01, 4.94910668e-03, 6.14228819e-03, 7.38169540e-03,
        8.64731809e-03],
       [9.54491227e-04, 4.97426885e-01, 4.22979340e-01, 3.57881752e-01,
        3.01218046e-01, 9.10035627e-03, 1.12826205e-02, 1.35447474e-02,
        1.58495559e-02],
       [1.55824690e-03, 4.49343559e-01, 3.80466522e-01, 3.20473994e-01,
        2.68465816e-01, 1.47820393e-02, 1.83067282e-02, 2.19524066e-02,
        2.56580382e-02],
       [2.33454339e-03, 4.01314003e-01, 3.38262822e-01, 2.83568945e-01,
        2.36357124e-01, 2.20292741e-02, 2.72505932e-02, 3.26385747e-02,
        3.8

In [118]:
 max([1, 2, -1], 0)

TypeError: '>' not supported between instances of 'int' and 'list'

In [121]:
comb(3, -1)

0.0

In [46]:
runGame()


These are your cards: [4, 2, 7, 2]
0.46022490628904633
[1, 0, 5, -1, 0.9995835068721368]
0.10037484381507707
Bluff


In [ ]:
dk


In [28]:
allCallProb = [onetype(state, numcards, 1, ownCards), onetype(state, numcards, 2, ownCards), twotype(state, numcards, 2, 2, ownCards), onetype(state, numcards, 3, ownCards), twotype(state, numcards, 3, 2, ownCards), onetype(state, numcards, 4, ownCards), twotype(state, numcards, 4, 3, ownCards)]
allCallProb

[array([0.82182144, 0.82182144, 0.82182144, 0.82182144, 0.82182144,
        0.82182144, 0.82182144, 0.82182144, 0.82182144]),
 array([0.48627712, 0.48627712, 0.48627712, 0.48627712, 0.48627712,
        0.48627712, 0.48627712, 0.48627712, 0.48627712]),
 array([[0.2012416, 0.2012416, 0.2012416, 0.2012416, 0.2012416, 0.2012416,
         0.2012416, 0.2012416, 0.2012416],
        [0.2012416, 0.2012416, 0.2012416, 0.2012416, 0.2012416, 0.2012416,
         0.2012416, 0.2012416, 0.2012416],
        [0.2012416, 0.2012416, 0.2012416, 0.2012416, 0.2012416, 0.2012416,
         0.2012416, 0.2012416, 0.2012416],
        [0.2012416, 0.2012416, 0.2012416, 0.2012416, 0.2012416, 0.2012416,
         0.2012416, 0.2012416, 0.2012416],
        [0.2012416, 0.2012416, 0.2012416, 0.2012416, 0.2012416, 0.2012416,
         0.2012416, 0.2012416, 0.2012416],
        [0.2012416, 0.2012416, 0.2012416, 0.2012416, 0.2012416, 0.2012416,
         0.2012416, 0.2012416, 0.2012416],
        [0.2012416, 0.2012416, 0.2012416

In [38]:
twotype(np.array([1, 2]), 4, 3, 2, np.array([3, 0]))


[[[0.5  0.5 ]
  [0.75 0.75]]

 [[0.5  0.75]
  [0.5  0.75]]]


array([[0.        , 0.        ],
       [0.20703125, 0.        ]])

In [30]:
np.array([1, 2])*np.array([2, 4])

array([2, 8])